# Semantics, Transformations, and Static Analysis

These notes cover relevant notions of semantics for programming languages, as well as common families of transformations associated with programming languages (such as interpretation, compilation, and transpilation). Static analysis concepts and techniques are also defined and discussed.

## Dependencies

In [73]:
# Presentation dependencies.
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib as mp
import matplotlib.pyplot as plt
from importlib import reload
from IPython.display import Image
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML

# Content dependencies (also reproduced inline).
from random import randint
from itertools import permutations
from functools import reduce
from tqdm import tqdm

## Denotational and Operational Semantics

The abstract syntax of a programming language is a set of symbolic objects (i.e., the abstract syntax instances, such as programs) that have no meaning unless a meaning is assigned to them. There are two ways in which we can assign meaning to these objects. We can choose to assign a mathematical object to each abstract syntax instance, or we can define a collection of deterministic transformations that specify how we can convert each abstract syntax instance into another abstract syntax instance. Roughly speaking, assigning a mathematical object to each program tells us what it *means*, while specifying symbolic converion rules tells us how to *run* the program by converting its syntactic constructs in a step-by-step manner into others.

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> The denotational semantics of an abstract syntax is a mapping from the set of abstract syntax instances $A$ to some mathematical set of objects $D$, which is often called a *semantic domain* or just *domain*. The mapping from $A$ to $D$ itself is often denoted using the circumfix Oxford double bracket notation $[[ ... ]]$, and the definition of a denotational semantics of $A$ (i.e., the definition of this mapping $[[ ... ]]$) is often specified using a collection of logical inference rules.
</div>

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> Let $A$ be an abstract syntax of a programming language. The operational semantics of an abstract syntax is a set of rules that specify how each abstract syntax instance $a \in A$ can be transformed some kind of object (possibly another element $a' \in A$) that represents the result of performing the computation (or individual computation step) described by $a$.
</div>

There are distinct kinds of operational semantics, such as *small-step semantics* and *big-step semantics* (also known as *natural semantics*). In these notes, the operational semantics we describe is closest to big-step semantics, with some simplifications. We adopt this particular approach to defining operational semantics because it corresponds more closely to a recursive implementation of an algorithm for interpreting ASTs.

The operational semantics for a programming language represents a *contract*, a set of *constraints*, or a set of *requirements* that an algorithm that implements an interpreter or compiler of that language must respect in order to be considered correct. However, whoever builds an implementation of an interpreter or compiler for a language has full freedom and flexibility in how they choose to implement the interpreter in all other aspects as long as it conforms to the operational semantics. This is what makes it possible to introduce optimizations into interpreters and compilers (such as optimizations to improve performance or reduce use of memory) while preserving the correctness of their behavior. The operational semantics for a programming language is defined using a collection of *inference rules*.

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> An <i>inference rule</i> is a notation used within mathematics and computer science to define relationships between mathematical facts and formulas. Each inference rule consists of a horizontal line with zero or more logical formulas above the line and one logial formula below the line. The logical formulas above the line are called *premises*, and the formula below the line is called the <i>conclusion</i>.
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\mathrm{\scriptsize [Name{-}of{-}Inference{-}Rule]} & \ & \frac{\ \ \textit{premise} \ \ \ \ \ \ \textit{premise}\ \ }{\textit{conclusion}}\\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\mathrm{\scriptsize [Example]} & \ & \frac{\ \ \textbf{sun is out} \ \ \ \ \ \ \textbf{sky is clear}\ \ }{\textbf{it is not raining}}\\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
An inference rule can be interpreted as a portion of a larger algorithm. The premises specify the recursive calls, or calls to other functions, that may need to be made, and the results that are obtained from those invocations. The conclusion specifies what inputs can be handled by that inference rule, and what outputs should be returned given those inputs and the premises.

Suppose we have a binary relation $\Downarrow$ between instances of an abstract syntax. We can read $a \Downarrow a'$ as <i>$a$ becomes $a'$</i> when we follow the operational semantics of the language*.
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\mathrm{\scriptsize [Algorithm{-}Case]} & \ & \frac{\ \ \textit{input}_1 \Downarrow \textit{output}_1 \ \ \ \ \ \ \textit{input}_1 \Downarrow \textit{output}_1\ \ }{\textit{input}_0 \Downarrow \textit{output}_0}\\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
Note that in the above, $\textit{input}_1$ and $\textit{input}_2$ may depend on $\textit{input}_0$, and $\textit{output}_0$ may depend on $\textit{output}_1$ and $\textit{output}_2$. In other words, one could rewrite an inference rule in the following way using natural language:
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\mathrm{\scriptsize [Algorithm{-}Case]} & \ & \frac{\textbf{this algorithm on } \textit{input}_1 \textbf{ yields } \textit{output}_1 \ \ \ \ \ \ \textbf{this algorithm on } \textit{input}_2 \textbf{ yields } \textit{output}_2}{\textbf{given } \textit{input}_0, \textbf{ if the above are true then output } \textit{output}_0 }\\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>

</div>

## Interpretation: Evaluation and Execution

The abstract syntax, or a subset of the abstract syntax, of a programming language is considered to be a set of *expressions* if the language's operational semantics do not impose any restrictions on the *order* in which a computation can operate on the expression to produce a result, called a *value*. This is possible because expressions usually represent operations with no *side effects* (such as emitting output to a screen, reading or writing files, looking at a clock, controlling a device, and so on).

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> Let $A$ be an abstract syntax of a programming language, and let $V \subset A$ be some subset of $A$ that we will call the <i>value set</i>. This set will represent the possible meanings of parse trees in $A$. Equivalently, it will represent the possible results of evaluating parse trees in $A$. Values that can occur directly within abstract syntax trees of the language (e.g., numeric and string literals, constructors, and so on) are usually called <i>constants</i>.
</div>

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> An <i>evaluation algorithm</i> converts any abstract syntax tree that represents an expression into an abstract syntax tree that represents a value.
<span style="height:25px; display:block;"></span>
$$\overbrace{a}^{\textbf{AST}} \underbrace{\longrightarrow}_{\textbf{evaluation}} \overbrace{v}^{\textbf{value}}$$
<span style="height:25px; display:block;"></span>
</div>

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<b>Example.</b> Define the abstract syntax according to the following grammar, with $A$ consisting of all formula abstract syntax instances, and $V = \{\textbf{true}, \textbf{true}\}$ consisting of all value abstract syntax instances:
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\textit{formula} & ::= & \textit{value} \ \ | \ \ \textbf{not} \ \textit{formula} \ \ | \ \
                         \textit{formula} \ \textbf{and} \ \textit{formula} \ \ | \ \ \textit{formula} \ \textbf{or} \ \textit{formula} \\
\textit{value} & ::= & \textbf{true} \ \ | \ \ \textbf{false} \\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
Let $\neg$, $\wedge$ and $\vee$ represent boolean negation, conjunction, and disjunction (respectively) on the values $\textbf{true}$ and $\textbf{false}$. Note that each of these is just a small truth table of two or four rows. The following is a definition of an operational semantics for this language. Because these operational semantics inference rules define how expressions can be converted into values, they also represent an evaluation algorithm.
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\mathrm{\scriptsize [Value]} & \ & \frac{}{v \Downarrow v}\\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\mathrm{\scriptsize [Not]} & \ & \frac{f \Downarrow v}{\textbf{not} \ f \Downarrow \neg v}\\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\mathrm{\scriptsize [And]} & \ & \frac{f_1 \Downarrow v_1 \ \ \ \ \ \ f_2 \Downarrow v_2}{f_1 \ \textbf{and} \ f_2 \Downarrow v_1 \wedge v_2}\\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
\begin{eqnarray*}
\mathrm{\scriptsize [Or]} & \ & \frac{f_1 \Downarrow v_1 \ \ \ \ \ \ f_2 \Downarrow v_2}{f_1 \ \textbf{or} \ f_2 \Downarrow v_1 \vee v_2}\\
\end{eqnarray*}
<span style="height:25px; display:block;"></span>
Note that this is not circular: the functions $\neg$, $\wedge$ and $\vee$ are much simpler than the operators $\textbf{not}$, $\textbf{and}$, and $\textbf{or}$ because the former can only operate on values (i.e., $\textbf{true}$ and $\textbf{false}$) while the latter can take arbitrary formulas as inputs.
</div>

In [88]:
import ast 

class evaluate_with_environment(ast.NodeVisitor):
    
    def visit_Module(self, node):
        results = []
        env = {}
        for s in node.body:
            s.env = env
        for s in node.body:
            results.append(self.visit(s))
        return results

    def visit_Assign(self, node):
        env = node.env
        node.value.env = env
        value = self.visit(node.value)
        env[node.targets[0].id] = value
    
    def visit_Expr(self, node):
        node.value.env = node.env
        return self.visit(node.value)

    def visit_BinOp(self, node):
        node.left.env = node.env
        node.right.env = node.env
        if type(node.op) is ast.Add:
            return self.visit(node.left) + self.visit(node.right)

    def visit_Num(self, node):
        return node.n

    def visit_Name(self, node):
        return node.env[node.id]

print(evaluate_with_environment().visit(ast.parse('8')))
print(evaluate_with_environment().visit(ast.parse('5 + 8 + 9 + 3')))
input = """
x = 123
y = 123 + x
y
"""
print(evaluate_with_environment().visit(ast.parse(input)))

[8]
[25]
[None, None, 246]


## Transformations

### Optimization

Suppose we want to create an optimization transformation that identifies subtrees of an AST that are *simple* (i.e., they consist only of numeric constants and operators), evaluates them into individual constants, and replaces the subtree with those constants.

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<a name="85feea29-7b81-4b58-bf08-bf8b0b2195a3"></a>
<b>Example [<a href="#85feea29-7b81-4b58-bf08-bf8b0b2195a3">link</a>].</b> Below is an implementation of an algorithm that identifies when a node in the AST represents an expression that contains only numeric literals and binary operators.
</div>

In [15]:
import ast 

class is_simple(ast.NodeVisitor):
    
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        return all(results)

    def visit_Expr(self, node):
        return self.visit(node.value)

    def visit_BinOp(self, node):
        return self.visit(node.left) and self.visit(node.right)
    
    def visit_Num(self, node):
        return True
    
a = ast.parse('5 + 5 - 7 // 4 * 4')
print(is_simple().visit(a))

a = ast.parse('x + 5')
print(is_simple().visit(a))

True
False


<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<a name="85feea29-7b81-4b58-bf08-bf8b0b2195b3"></a>
<b>Example [<a href="#85feea29-7b81-4b58-bf08-bf8b0b2195b3">link</a>].</b> Below is an implementation of an optimization algorithm that evaluates all simple expressions (as determined by the algorithm in the <a href="#85feea29-7b81-4b58-bf08-bf8b0b2195a3">example</a> above).
</div>

In [16]:
import astor

class optimize(ast.NodeTransformer):

    def visit_Expr(self, node):
        check = is_simple().visit(node)
        print("Checking: " + str(check))
        n = eval(astor.to_source(node))
        a =\
          ast.copy_location(
            ast.Num(n),
            node
          )
        return a

a = ast.parse('5 + 5 - 7 // 4 * 4')
ast.dump(a)
print(ast.dump(a))
a = optimize().visit(a)
ast.dump(a)

Module(body=[Expr(value=BinOp(left=BinOp(left=Num(n=5), op=Add(), right=Num(n=5)), op=Sub(), right=BinOp(left=BinOp(left=Num(n=7), op=FloorDiv(), right=Num(n=4)), op=Mult(), right=Num(n=4))))])
Checking: True


'Module(body=[Num(n=6)])'

## Static Analysis and Abstract Interpretation

One important distinction between interpretation algorithms and static analysis algorithms is that static analysis algorithms must terminate (typically in a very short amount of time). One reason is that static analysis techniques such as type checking must run during compilation.

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<a name="85feea29-7b81-4b58-bf08-bf8b0b2195e3"></a>
<b>Example [<a href="#85feea29-7b81-4b58-bf08-bf8b0b2195e3">link</a>].</b> Below is an implementation of a static analysis algorithm that infers the type of a single Python expression.
</div>

In [11]:
import ast 

class infer_type(ast.NodeVisitor):
    
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        if results == [int] or results == [bool]:
            return results[0]

    def visit_Expr(self, node):
        return self.visit(node.value)

    def visit_BoolOp(self, node):
        if set([self.visit(v) for v in node.values]) == {bool}:
            return bool

    def visit_BinOp(self, node):
        if type(node.op) in [ast.Add, ast.Sub, ast.Mult]:
            if self.visit(node.left) is int and\
               self.visit(node.right) is int:
                return int
    
    def visit_Compare(self, node):
        if type(node.ops[0]) in [ast.Eq]:
            if self.visit(node.left) is int and\
               self.visit(node.comparators[0]) is int:
                return bool
    
    def visit_Num(self, node):
        if type(node.n) is int:
            return int
    
    def visit_NameConstant(self, node):
        if node.value in [True, False]:
            return bool
    
print(infer_type().visit(ast.parse('5')))
print(infer_type().visit(ast.parse('5 + 5')))
print(infer_type().visit(ast.parse('True and False')))
print(infer_type().visit(ast.parse('False or 5')))
print(infer_type().visit(ast.parse('False + True')))
print(infer_type().visit(ast.parse('3 or 4 and 5')))
print(infer_type().visit(ast.parse('3 == 5')))
#print(infer_type().visit(ast.parse('5')))


<class 'int'>
<class 'int'>
<class 'bool'>
None
None
None
<class 'bool'>


<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
<a name="85feea29-7b81-4b58-bf08-bf8b0b2195e5"></a>
<b>Example [<a href="#85feea29-7b81-4b58-bf08-bf8b0b2195e5">link</a>].</b> The static analysis algorithm below uses the <a href="https://www.sympy.org/en/index.html">SymPy</a> library to build a polynomial that represents the number of digits in the integer represented by a Python expression. Notice that if the expression contains variables, the polynomial will have placeholders for the number of digits that the integer represented by the variable may have.
</div>

In [46]:
import ast
from sympy import sympify, lambdify

class infer_int_digits(ast.NodeVisitor):
    
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        return results

    def visit_Expr(self, node):
        return self.visit(node.value)

    def visit_BinOp(self, node):
        if type(node.op) in [ast.Sub]:
            e1 = self.visit(node.left)
            e2 = self.visit(node.right)
            return e1
        elif type(node.op) in [ast.Mult]:
            e1 = self.visit(node.left)
            e2 = self.visit(node.right)
            return e1 + e2 + 1

    def visit_Name(self, node):
        return sympify(node.id + "_digits")

    def visit_Num(self, node):
        if type(node.n) is int:
            digits = len(str(node.n))
            return sympify(str(digits))

result = infer_int_digits().visit(ast.parse('x * x * 45 * y - z * 10000'))[0]
f = lambdify('f', result)
f((1,2))

2*x_digits + y_digits + 5

## Embedding Techniques for Transformations and Analyses

Suppose we have assembled some formal definitions of an embedded language:

* some BNF notation describing the subset of the syntax of our host language,
* some inference rules for static analysis algorithms over that subset of the syntax, and
* some inference rules for transformation algorithms over that subset of the syntax.

How can we approach implementing these within the host language? We can use <i>reflection</i>.

<div style="background-color:#F6F4D8; padding:5px 8px 5px 8px;">
<b>Definition.</b> The term <i>reflection</i> refers to the <i>native</i> (i.e., built-in) capability within a programming language to name, store, examine, modify, and create programs <i>written in that same language</i>.
</div>

We have already seen how the `inspect` and `ast` modules can enable reflection in Python. However, we may want to make it easy for programmers to utilize the algorithms we have defined over our embedded language. Python [decorators](https://www.python.org/dev/peps/pep-0318/) are one way to quickly build an API for our embedded language. One popular library that uses decorators to provide a convenient API is [Flask](https://en.wikipedia.org/wiki/Flask_(web_framework)).

In [4]:
# Turns any function of one variable into a function 
# that returns the same result, but also displays it.
def make_print(h):
    def h_prints(x):
        result = h(x)
        print("The result is: " + str(result))
        return result
    return h_prints

@make_print
def f(x):
    return x + x

f(4)

The result is: 8


8

In [7]:
# This decorator takes an argument (in other words,
# it is a function that returns a decorator).
def make_print_decorator(message):
    def decorator(h):
        def h_prints(x):
            result = h(x)
            print(message + ": " + str(result))
            return result
        return h_prints
    return decorator

@make_print_decorator("The result is")
def f(x):
    return x + x

f(4)

The result is: 8


8

In [10]:
import inspect
import ast

# Decorator that prints the source code of
# the function that it decorates.
def print_source_opt(visible = True):
    if not visible:
        return lambda f: f
    else:
        def print_source(f):
            a = ast.parse(inspect.getsource(f))
            print(ast.dump(a))
            return f
        return print_source

# This will not print anything.
@print_source_opt(False)
def g(x):
    return 2 * x

# This will print the AST.
@print_source_opt(True)
def h(x):
    return 3 + x
   

Module(body=[FunctionDef(name='h', args=arguments(args=[arg(arg='x', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Num(n=3), op=Add(), right=Name(id='x', ctx=Load())))], decorator_list=[Call(func=Name(id='print_source_opt', ctx=Load()), args=[NameConstant(value=True)], keywords=[])], returns=None)])


In [33]:
import inspect
import ast

# Decorator that prints the source code of
# the function that it decorates.
def print_source_opt(visible = True):
    if not visible:
        return lambda f: f
    else:
        def print_source(f):
            a = ast.parse(inspect.getsource(f))
            print(ast.dump(a))
            return f
        return print_source

# This will not print anything.
@print_source_opt(False)
def g(x):
    return 2 * x

# This will print the AST.
@print_source_opt(True)
def h(x):
    return 3 + x
   

Module(body=[FunctionDef(name='h', args=arguments(args=[arg(arg='x', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=[Return(value=BinOp(left=Num(n=3), op=Add(), right=Name(id='x', ctx=Load())))], decorator_list=[Call(func=Name(id='print_source_opt', ctx=Load()), args=[NameConstant(value=True)], keywords=[])], returns=None)])


This is the end of the document.